In [1]:
import cv2
import os

In [2]:
print(cv2.__version__)

4.9.0


In [3]:
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [4]:

filepath = './C_3_9_1_BU_DYA_08-02_13-36-38_CA_RGB_DF2_M2.mp4'


try:
    video = cv2.VideoCapture(filepath)
    if not video.isOpened():
        raise Exception("비디오 파일을 열 수 없습니다.")
except Exception as e:
    print("오류:", str(e))
    exit(0)


In [5]:
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)

print("length :", length)
print("width :", width)
print("height :", height)
print("fps :", fps)


length : 180
width : 1920
height : 1080
fps : 3.0


In [6]:
try:
    if not os.path.exists(filepath[:-4]):
        os.makedirs(filepath[:-4])
except OSError:
    print ('Error: Creating directory. ' +  filepath[:-4])

In [ ]:
count = 0

while(video.isOpened()):
    ret, image = video.read()
    if(int(video.get(1)) % fps == 0): #앞서 불러온 fps 값을 사용하여 1초마다 추출
        cv2.imwrite(filepath[:-4] + "/frame%d.jpg" % count, image)
        print('Saved frame number :', str(int(video.get(1))))
        count += 1
        
video.release()